# Machine Learning Technologies Assignment

### Import Libraries

In [1]:
import numpy as np
import nltk
import string
import re
from dateutil.parser import parse

import matplotlib.pyplot as plt

import pandas as pd
pd.set_option('display.max_colwidth', 100)

import spacy

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

##Model Library

### Variable Initialization

### Load Data

In [24]:
f = open('Dataset/mediaeval-2015-trainingset.txt', encoding = "ISO-8859-1").read().split('\n')
cols = f[0].split('\t')
f_data = []
for tweet in f:
    t = tweet.split('\t')
    f_data.append(t)
raw_df = pd.DataFrame(f_data, columns=cols)
raw_df = raw_df.iloc[1:]
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14483 entries, 1 to 14483
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweetId     14483 non-null  object
 1   tweetText   14483 non-null  object
 2   userId      14483 non-null  object
 3   imageId(s)  14483 non-null  object
 4   username    14483 non-null  object
 5   timestamp   14483 non-null  object
 6   label       14483 non-null  object
dtypes: object(7)
memory usage: 792.2+ KB


In [25]:
raw_df.head()

,tweetId,tweetText,userId,imageId(s),username,timestamp,label
1,263046056240115712,Â¿Se acuerdan de la pelÃ­cula: âEl dÃ­a despuÃ©s de maÃ±anaâ? Me recuerda a lo que estÃ¡ pas...,21226711,sandyA_fake_46,iAnnieM,Mon Oct 29 22:34:01 +0000 2012,fake
2,262995061304852481,"@milenagimon: Miren a Sandy en NY! Tremenda imagen del huracÃ¡n. Parece el ""DÃ­a de la Independ...",192378571,sandyA_fake_09,CarlosVerareal,Mon Oct 29 19:11:23 +0000 2012,fake
3,262979898002534400,"Buena la foto del HuracÃ¡n Sandy, me recuerda a la pelÃ­cula DÃ­a de la Independencia #ID4 #Sand...",132303095,sandyA_fake_09,LucasPalape,Mon Oct 29 18:11:08 +0000 2012,fake
4,262996108400271360,Scary shit #hurricane #NY http://t.co/e4JLBUfH,241995902,sandyA_fake_29,Haaaaarryyy,Mon Oct 29 19:15:33 +0000 2012,fake
5,263018881839411200,My fave place in the world #nyc #hurricane #sandy #statueofliberty ð½ http://t.co/Ex61doZk,250315890,sandyA_fake_15,princess__natt,Mon Oct 29 20:46:02 +0000 2012,fake


### Extract features from timestamp

In [ ]:
raw_df['timestamp'] = raw_df['timestamp'].